Author: Bryn Nurczyk

*Example code for cloud computing with Google Earth Engine (GEE) in the derivation of indices and/or spectral analysis (Landsat, Sentinel-2, PlanetScope)*

**Returns:** a multi-band .tif/.csv file of vegetation indices, FVC, LST, and Tasseled Cap Transformation; \
additional modules for PCA; \
Time series and plotting

### Citations:

Ermida, S.L., Soares, P., Mantas, V., Göttsche, F.-M., Trigo, I.F., 2020. 
    Google Earth Engine open-source code for Land Surface Temperature estimation from the Landsat series.
    Remote Sensing, 12 (9), 1471; https://doi.org/10.3390/rs12091471



In [2]:
import ee
import geemap
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import requests
from requests.auth import HTTPBasicAuth
import os, sys, importlib
import planet
import geopandas as gpd
from requests.auth import HTTPBasicAuth

from planet.subscription_request import (
    toar_tool,
    reproject_tool,
    file_format_tool,
    build_request,
    catalog_source,
)

In [5]:
ee.Authenticate()
ee.Initialize(project='seer-hl')


In [6]:

os.environ['PL_API_KEY'] = 'PLAKad40a40b5693493ea8e31256760dfc8f' 
print(os.getenv("PL_API_KEY"))

PLAKad40a40b5693493ea8e31256760dfc8f


Define appropiate variables

In [7]:
date_start = ee.Date('2019-01-01')
date_end   = ee.Date('2019-01-31')

aoi_name = 'example'

# aoi_path = "azone_subset1" 
# Define path of shapefile (INCLUDE ALL NESSECARY FILES!!) -- Only necessary if AOI shp 
# is not uploaded as a feature collection in Google Earth Engine project assets.
aoi_fc = ee.FeatureCollection('projects/seer-brennurczyk/assets/Azone_subset1')
aoi_path =  "/blue/jkblackburn/bnurczyk/Azone/Azone.gpkg"

export_to = 'example'
crs = 'EPSG:26914'


"""
Choose a satellite system
Landsat 4–9; 'L4','L5','L7','L8','L9'
Sentinel-2; 'S2'
PlanetScope: 'PS'
Harmonization handled separately
"""
satellite = 'PS'


"""
Define parse-by temporal step as string:
- Monthly: 'M'
- Annual:  'A'
- Seasonal: default '-'
- None (use entire timespan as aggregate): 'N'
"""
step = 'M'

summary_tif='median'

gee_img_collection='planet-test'


In [5]:
import importlib, modules
importlib.reload(modules)
from modules import parse
importlib.reload(parse)
from plaknit import planner as planner_cli
#geojson_geometry = (aoi.union().geometry() if isinstance(aoi, ee.FeatureCollection) else aoi).getInfo()

# Function: 
# retrieves imgaery from GEE or, for PS imagery, a Planet Quicksearh
# Returns: 
# a dictionary of GEE image collections and for Planetscope, GEOJsons. PS orders will be placed within the analysis step (TOADict, SRDict will return empty)
cloudDict, TOADict, SRDict = parse.retrieveImagery(satellite, date_start, date_end, step, aoi_fc, aoi_path)


Scene 20190123_185659_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185659_1_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185658_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185657_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185657_1_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185656_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185656_1_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185655_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185654_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185654_1_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185653_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185652_1_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185652_0f1c is missing clear/cloud metadata; skipping.
Scene 20190123_185651_0f1c is missing clear/cloud metadata; skipping.
Scene 2019

In [11]:
import modules
importlib.reload(modules)

from modules import ndvi, cloudmask, main, evi, savi, tct
importlib.reload(ndvi); importlib.reload(cloudmask); importlib.reload(main); importlib.reload(evi); importlib.reload(savi); importlib.reload(tct)


collectedDict = await main.collection(
    satellite, date_start, date_end, step, aoi_fc, aoi_path, SRDict, TOADict, cloudDict,
    cloudmask_sr=cloudmask.sr,
    ndvi_addBand=ndvi.addBand,
    evi_addBand=evi.addBand,
    savi_addBand=savi.addBand,
    tct_addBands=tct.addBands
)

2024-01: 136 of 136 have SR
Submitting Planet order for 2024-01 (136 scenes)


1:13:52 - order ba107923-6d6f-4aeb-952d-eff306ff0c1b - state: failed 


In [17]:
import ee
import geemap
from modules import ndvi
import importlib
importlib.reload(ndvi)
from geemap.foliumap import Map
from modules import ndvi
import importlib
importlib.reload(ndvi)

geemap.ee_initialize()

def _add_ndvi(img):
    return ndvi.addBand(img, satellite)

col = ee.ImageCollection("projects/seer-hl/assets/planet-test");

planet = (
    col
      .filter(ee.Filter.stringContains('system:index', '_AnalyticMS_SR'))
      .map(lambda img: img.clip(aoi_fc))
)


ic = (
    planet
      .map(_add_ndvi)
      .select('NDVI')
)

ndvi_median = ic.median().clip(aoi_fc)

ndvi_vis = {
    'min': 0.0,
    'max': 0.85,
    'palette': [
        '#654321',  # bare soil
        '#a97142',
        '#d9b382',
        '#efe6c9',
        '#c7e9c0',
        '#7fcdbb',
        '#41ab5d',
        '#238b45',
        '#006d2c'   # dense vegetation
    ]
}


m = Map()
m.centerObject(aoi_fc, 10)
m.addLayer(ndvi_median, ndvi_vis, 'PlanetScope NDVI (median)')
rgb_vis = {
    'bands': ['B3', 'B2', 'B1'],  # R, G, B
    'min': 0,
    'max': 3000
}

m.addLayer(planet, rgb_vis, 'PlanetScope RGB')

m.addLayer(aoi_fc, {}, 'AOI')
m
